## 获取头条街拍图片并保存

In [1]:
import requests 
#1.发送请求
#请求头
#伪装成浏览器请求的形式， 否则有可能报400 bad request的报错
#需要使用以下headers信息，否则无法拿到图集信息
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 Edg/87.0.664.66',
    'authority': 'www.toutiao.com',
    'method': 'GET',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'cookie': 'tt_webid=6911225535029626381; ttcid=7e4bb598b68645f790150af8b4975d6168; s_v_web_id=verify_kj8bfyb7_zLTGxvH1_yKQ2_4ZIo_9SRg_P9VHi8KUv7pj; csrftoken=a49abea0d92ce7f2558511705d42da93; tt_webid=6911225535029626381; __tasessionId=3s0w4doeb1609145109505; csrftoken=a49abea0d92ce7f2558511705d42da93; __ac_nonce=05fe99b2700f27d85f9d7; __ac_signature=_02B4Z6wo00f01vWsitwAAIBCTWr0gxUCUo71qI5AAOKQbQxYUl2qMLCYwgLhbyl6F-sI5-FFl6AXekzCTVeH6wjoVDf1u21GyD23r5dLKR3UizA.KCBu99z07avq1ojY3HSrRRH7ZYnRq14s56; MONITOR_WEB_ID=c26adb0f-65b0-4f72-b0f7-2fc5ae75ffe5; tt_scid=MpdvuOnHHFKinr9nfzbvWsh0eXD09.CRbyFVsCtSYDpxpHqHsnvMmCXjef.QWjdUb1ef',
    #'referer': 'https://www.toutiao.com/a6817621907341312515/'
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1'
}

In [2]:
from urllib.parse import urlencode

In [3]:
def get_page_index(offset, keyword):
    data = {
        'aid': 24,
        'app_name': 'web_search',
        'offset': offset,
        'format': 'json',
        'keyword': keyword,
        'autoload': 'true',
        'count': 20,
        'en_qc': 1,
        'cur_tab': 1,
        'from': 'search_tab',
        'pd': 'synthesis',
        'timestamp': 1609145232620,
        '_signature': '_02B4Z6wo00f01eA4y2gAAIBBWP61NwctdFXgPc.AACfqDI8Q9lOhD5ochrJKWAUYj5-VBKFXYV1yZyzs7xtFv7qq4yj0vgUFwhE4TiKGH9W-O.kMXFBSLAKb-234dLDGYZ45gGpgWZp2eC2t6d'
    }
    
    url = 'https://www.toutiao.com/api/search/content/?' + urlencode(data)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        print('请求索引失败')
        return None
    
    #print(url)

In [4]:
import json

In [5]:
#从拿到的索引页信息中获取图集的url
def parse_page_index(html):
    data = json.loads(html)
    if data and 'data' in data.keys(): #判断json中是否含有data这个键值，同时也判断html是否成功拿到
        for item in data.get('data'):
            yield item.get('article_url')

In [6]:
def get_page_detail(url):
    try:
        response = requests.get(url, headers = headers)
        if response.status_code == 200:
            return response.text #返回一个html格式的响应
        return None
    except RequestException:
        print('请求详情页失败'+url)
        return None

In [7]:
from bs4 import BeautifulSoup

In [8]:
#返回一个list，包含了某个图集里面所有的图片url
def parse_page_detail(html):
    lst = []
    soup = BeautifulSoup(html, 'lxml')
    result = soup.find_all(attrs={'class':'image-item'}) #获得一个set集合,包含了图片的url
    for item in result:
        if item.img['src'] is not None:
            #print('img [src]'+item.img['src'])
            lst.append(item.img['src']) #把图片的url放到列表里
    
    if not result: #不能使用是否为None判断，空列表可以用if not
        result = soup.find_all(attrs={'class':'pgc-img'}) #这里发现有的图集会把图片存在这个标签下面
        for item in result:
            if item.img['src'] is not None:
                #print('img [src]'+item.img['src'])
                lst.append(item.img['src']) #把图片的url放到列表里
    
    return lst

In [9]:
html = get_page_index(0, '街拍')
picture_lst = []
for url in parse_page_index(html):
    if url is not None and 'toutiao' in url: #只选取头条内部的图集，所以筛选包含toutiao字符的网址
        url = url.replace("group/", "a") #这里要做修改因为我们从json拿到的url和图集实际所在的url有一点区别，要把group/替换为a
        url = url.replace("://","s://www.")
        print(url)
        html = get_page_detail(url)
        #print(html) 
        picture_url = parse_page_detail(html) #返回的是一个list
        #print(picture_url)
        picture_lst.extend(picture_url)
    

https://www.toutiao.com/a6817621907341312515/
https://www.toutiao.com/a6609426526892982798/
https://www.toutiao.com/a6881280526875886093/
https://www.toutiao.com/a6583975198469390861/
https://www.toutiao.com/a6428768938196206081/
https://www.toutiao.com/a6789555264598049284/
https://www.toutiao.com/a6874741872464495118/
https://www.toutiao.com/a6865502980410442254/
https://www.toutiao.com/a6797025796830528004/
https://www.toutiao.com/a6870167456942391811/
https://www.toutiao.com/a6890911912192639499/
https://www.toutiao.com/a6911154539265573379/
https://www.toutiao.com/a6859294282008920584/


In [10]:
picture_lst #得到图片的url集合

['https://p3-tt.byteimg.com/origin/pgc-image/53eeb158296541c09fa04c1421cb9810?from=pc',
 'https://p3-tt.byteimg.com/origin/pgc-image/eeec754ae3f8490989524e2d9e4788a6?from=pc',
 'https://p3-tt.byteimg.com/origin/pgc-image/de40ae4392d348b684da34ba9967326b?from=pc',
 'https://p6-tt.byteimg.com/origin/pgc-image/1566c305a5064569a4cc97560d4afec7?from=pc',
 'https://p3-tt.byteimg.com/origin/pgc-image/24c8db9cad5f4798997fc1ef92766972?from=pc',
 'https://p1-tt.byteimg.com/origin/pgc-image/565e63bfdebf419a90b0266cda71757b?from=pc',
 'https://p1-tt.byteimg.com/origin/pgc-image/7e7a0d35393e4ef7884ff5c997a5aeed?from=pc',
 'https://p3-tt.byteimg.com/origin/pgc-image/c2dcf2c64e534e9897ca10f1df3dd7be?from=pc',
 'https://p1-tt.byteimg.com/origin/pgc-image/8a62313ac7cb49e1890bd089cfd99abe?from=pc',
 'https://p1-tt.byteimg.com/origin/pgc-image/a49fceb2598a47f9a8d4b6d519eb7719?from=pc',
 'https://p1-tt.byteimg.com/origin/pgc-image/5e20837b2b8c4b3b8149c12f5d5c014d?from=pc',
 'https://p3-tt.byteimg.com/orig

In [20]:
import openpyxl

In [39]:
def writeToExcel(file_path, new_list):
        # total_list = [['A', 'B', 'C', 'D', 'E'], [1, 2, 4, 6, 8], [4, 6, 7, 9, 0], [2, 6, 4, 5, 8]]
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = '街拍'
        for r in range(len(new_list)):
            ws.cell(r + 1, 1).value = new_list[r]
            # excel中的行和列是从1开始计数的，所以需要+1
        wb.save(file_path)  # 注意，写入后一定要保存
        print("成功写入文件: " + file_path + " !")
        return 1

In [40]:
writeToExcel('街拍.xls',picture_lst)

成功写入文件: 街拍.xls !


1